# Orbital Decay Time Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('space_decay_wpca.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14125 entries, 0 to 14124
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   OBJECT_NAME            14125 non-null  object 
 1   OBJECT_ID              14125 non-null  object 
 2   EPOCH                  14125 non-null  object 
 3   INCLINATION            14125 non-null  float64
 4   RA_OF_ASC_NODE         14125 non-null  float64
 5   ARG_OF_PERICENTER      14125 non-null  float64
 6   MEAN_ANOMALY           14125 non-null  float64
 7   NORAD_CAT_ID           14125 non-null  int64  
 8   REV_AT_EPOCH           14125 non-null  int64  
 9   BSTAR                  14125 non-null  float64
 10  MEAN_MOTION_DOT        14125 non-null  float64
 11  MEAN_MOTION_DDOT       14125 non-null  float64
 12  OBJECT_TYPE            14125 non-null  int64  
 13  RCS_SIZE               14125 non-null  int64  
 14  COUNTRY_CODE           14125 non-null  object 
 15  LA

## Feature Selection

> In the context of predicting orbital decay, I initially performed a mutual information (MI) analysis to measure the dependency between each feature and the target variable. MI is a non-parametric method that captures both linear and nonlinear relationships between variables. The following features were selected based on their relatively high mutual information scores, indicating that they each contribute unique and significant information regarding the decay behavior of satellites.

'BSTAR','CROSS_SECTIONAL_AREA','MEAN_MOTION_DOT', 'DRAG_EFFECTIVE_AREA','APOPERI_RATIO','REV_AT_EPOCH','INCLINATION', (PC1,PC2)

In [5]:
from sklearn.feature_selection import RFE
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score

In [8]:
X = df.drop(columns=['ORBITAL_DECAY_TIME','EPOCH','OBJECT_ID','OBJECT_NAME','COUNTRY_CODE','SITE','ESTIMATED_DECAY_EPOCH'])
Y = df['ORBITAL_DECAY_TIME']

model = CatBoostRegressor(verbose=0, n_estimators=100)

cv_scores = []
feature_counts = range(1, X.shape[1] + 1)
selected_features_list = []
feature_rankings_list = []

for n_features in feature_counts:
    rfe = RFE(estimator=model, n_features_to_select=n_features)
    X_reduced = rfe.fit_transform(X, Y)

    scores = cross_val_score(model, X_reduced, Y, cv=4, scoring='neg_mean_squared_error')
    cv_scores.append(np.mean(scores))

    selected_features_list.append(X.columns[rfe.support_])
    feature_rankings_list.append(rfe.ranking_)

optimal_features = feature_counts[np.argmax(cv_scores)]
optimal_selected_features = selected_features_list[np.argmax(cv_scores)]
optimal_feature_rankings = feature_rankings_list[np.argmax(cv_scores)]

print(f"Optimal number of features: {optimal_features}")
print(f"Selected features: {list(optimal_selected_features)}")
print(f"Feature rankings: {optimal_feature_rankings}")

Optimal number of features: 5
Selected features: ['BSTAR', 'MEAN_MOTION_DOT', 'APOPERI_RATIO', 'DRAG_EFFECTIVE_AREA', 'PC2']
Feature rankings: [ 6 11  5  3  9  4  1  1  2 10 13 12  1  8  1  7  1]


> To ensure both statistical relevance and predictive performance in modeling orbital decay time, I performed feature selection using Recursive Feature Elimination (RFE) with a CatBoost regressor, a gradient boosting model known for handling non-linear relationships and categorical features effectively. While any machine learning model could be employed for feature selection, CatBoost was chosen for its robustness and interpretability in tabular data. The model identified five optimal features which consistently yielded the lowest mean squared error across cross-validation folds. Notably, all selected features also demonstrated high mutual information with the target variable, confirming their strong statistical dependency and physical relevance in the orbital decay process. Therefore, moving forward, these five features will be used as the core input set for model development and evaluation.

'BSTAR', 'MEAN_MOTION_DOT', 'APOPERI_RATIO', 'DRAG_EFFECTIVE_AREA', 'PC2'

## Model Training

In [9]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import autogluon.tabular as ag

/home/sinem/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
x = df[list(optimal_selected_features)]
y = df['ORBITAL_DECAY_TIME']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

models = {
    'RandomForestRegressor': (RandomForestRegressor(random_state=42), {
        'n_estimators': [100],
        'max_depth': [10],
    }),
    'GradientBoostingRegressor': (GradientBoostingRegressor(random_state=42), {
        'n_estimators': [100],
        'learning_rate': [0.1],
    }),
    'AdaBoostRegressor': (AdaBoostRegressor(random_state=42), {
        'n_estimators': [100],
        'learning_rate': [0.1],
    }),
    'LGBMRegressor': (LGBMRegressor(random_state=42), {
        'n_estimators': [100],
        'learning_rate': [0.1],
        'max_depth': [10],
    }),
    'CatBoostRegressor': (CatBoostRegressor(verbose=0, random_state=42), {
        'iterations': [100],
        'learning_rate': [0.1],
        'depth': [6],
    }),
}
results = []

for model_name, (model, param_grid) in models.items():
    print(f"Running {model_name}...")

    # Use GridSearchCV to find the best hyperparameters
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='r2')
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    train_score = best_model.score(X_train, y_train)
    test_score = best_model.score(X_test, y_test)

    results.append({
        'model': model_name,
        'best_params': grid.best_params_,
        'train_score': train_score,
        'test_score': test_score
    })

if os.path.exists('results.json'):
    try:
        with open('results.json', 'r') as f:
            previous_results = json.load(f)
    except json.JSONDecodeError:
        previous_results = []
else: 
    previous_results = []

# Append the new results to the previous results
previous_results.extend(results)
with open('results.json', 'w') as f:
    json.dump(previous_results, f, indent=4)


Running RandomForestRegressor...
Running GradientBoostingRegressor...
Running AdaBoostRegressor...
Running LGBMRegressor...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001367 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 7533, number of used features: 5
[LightGBM] [Info] Start training from score 9969.016195
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

> Based on the upper code Catboost is the best model so I will try some other parameters for catboost

"model": "CatBoostRegressor",
        "best_params": {
            "depth": 6,
            "iterations": 100,
            "learning_rate": 0.1
        },
        "train_score": 0.9887511930392616,
        "test_score": 0.7886693875698086

In [26]:
import itertools

# Features and target
x = df[list(optimal_selected_features)]
y = df['ORBITAL_DECAY_TIME']

filename = 'results_cb.json'

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define only CatBoostRegressor with hyperparameter grid
model_params = {
    'CatBoostRegressor': (CatBoostRegressor(verbose=0, random_state=42), {
        'iterations': [100, 200],
        'learning_rate': [ 0.05, 0.1],
        'depth': [4],
        'l2_leaf_reg': [3],
        'border_count': [32]
    }),
}

# Iterate over models and parameter grids
for model_name, (model_base, param_grid) in model_params.items():
    print(f"Running {model_name}...")

    combinations = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())

    results = []

    for params in combinations:
        param_dict = dict(zip(param_keys, params))
        model = model_base.__class__(**param_dict, verbose=0, random_state=42)

        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        results.append({
            'model': model_name,
            'params': param_dict,
            'train_score': train_score,
            'test_score': test_score
        })

# Save all results at once
with open(filename, 'w') as json_file:
    json.dump(results, json_file, indent=4)

Running CatBoostRegressor...


### 📊 CatBoostRegressor Results

| Iterations | Learning Rate | Depth | L2 Leaf Reg | Border Count | Train Score | Test Score |
|------------|----------------|-------|--------------|----------------|--------------|-------------|
| 100        | 0.05           | 4     | 3            | 32             | 0.5754       | 0.5630      |
| 100        | 0.1            | 4     | 3            | 32             | 0.6498       | 0.6462      |
| 200        | 0.05           | 4     | 3            | 32             | 0.6492       | 0.6464      |
| 200        | 0.1            | 4     | 3            | 32             | 0.7155       | 0.7051      |


In [45]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

df_sorted = df.sort_values(by='EPOCH')

X = df_sorted[list(optimal_selected_features)]
y = df_sorted['ORBITAL_DECAY_TIME']

# Manual time-based validation split (first 20% for validation)
n = len(df_sorted)
val_end = int(n * 0.2)

X_val, y_val = X.iloc[:val_end], y.iloc[:val_end]

# The rest (80%) is for train-test split
X_remaining = X.iloc[val_end:]
y_remaining = y.iloc[val_end:]

# Split the remaining 80% into 70% train and 30% test (adjust as needed)
X_train, X_test, y_train, y_test = train_test_split(X_remaining, y_remaining, test_size=0.25, random_state=42)

# Model with fixed hyperparameters
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.1,
    depth=4,
    l2_leaf_reg=3,
    border_count=32,
    verbose=0,
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test)

# Metrics function
def print_metrics(name, y_true, y_pred):
    r2 = model.score(X_train if name == "Train" else X_val if name == "Validation" else X_test, y_true)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"\n{name} Metrics:")
    print(f"  R²:   {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")

# Print metrics
print_metrics("Train", y_train, y_pred_train)
print_metrics("Validation", y_val, y_pred_val)
print_metrics("Test", y_test, y_pred_test)


Train Metrics:
  R²:   0.7420
  RMSE: 181.9555
  MAE:  19.3913

Validation Metrics:
  R²:   0.5155
  RMSE: 559.6122
  MAE:  67.4432

Test Metrics:
  R²:   0.1896
  RMSE: 379.4062
  MAE:  39.1564


> The model demonstrates a reasonable fit on the training set with an R² of 0.7420, indicating that it can explain approximately 74.2% of the variance in the target variable, accompanied by a relatively low RMSE and MAE. However, performance significantly drops on the validation and test sets, with R² scores of 0.5155 and 0.1896, respectively. This decline suggests that the model may be overfitting the training data and struggling to generalize to unseen samples, particularly under time-based validation, where temporal distribution shifts can challenge model robustness. The large increase in RMSE and MAE on the validation and test sets further supports this, highlighting the need for improved generalization, possibly through regularization or re-examination of temporal patterns in the dataset.

> The current project requires further development to enhance its overall quality. It is suggested that time series analyses be implemented to refine the methodology. Please provide guidance on how to rectify any errors present in the work.